In [ ]:
#Importing all libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import re
import time
#To display the last output not only the last result we use interactive shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### City names and Codes

Delhi=2,
Mumbai=2378, 
Bangalore=4709,
Hyderabad=3686,
Kolkata=7777,
Chennai=5732,
Gurgaon=5,
Ahmadabad=1692,
Noida=134,
Pune=2423

# Webscraping from Cars 24 website

In [ ]:
# To append the data from website empty lists are needed
productname = []
Res=[]
Ty=[]
range1=[]
price = []
paym=[]
Bran=[]
pagenum=[]
loc = []




Cities={2:'Delhi',2378:'Mumbai',4709:'Bangalore',3686:'Hyderabad',7777:'Kolkata',5732:'Chennai',5:'Gurgaon',1692:'Ahmadabad',134:'Noida',2423:'Pune'}

for j,k in Cities.items():
    
    for i in range(1,21):
        start_time = time.time()
        URL ='https://www.cars24.com/buy-used-car/?sort=P&page={}&storeCityId={}'.format(i,j)
        print(URL)
        page = requests.get(URL,verify=False)
        pagecontent = page.text
        
        soup = BeautifulSoup(pagecontent)
    
        
       
        for x in soup.find_all('div', attrs={'class':'col-4'}):
            
            
           
            pname = x.find('h2', attrs = {'class':'_3FpCg'})
            Reserved=x.find('div',attrs={'class':'_23Bx7'})
            Type=x.find('p',attrs={'class':'cvakB'})
            km=x.find('ul', attrs = {'class':'bVR0c'})
            cost=x.find('div', attrs = {'class':'_19zc5'})
            payment=x.find('span',attrs={'class':'_3X7mV _2H07I'})
            Brand=x.find('div',attrs={'class':'card _1DywR'})
                
           
            loc.append(k)
            
            if pname is None:
                productname.append(np.NaN)
            else:
                productname.append(pname.text)
                
            if Reserved is None:
                Res.append(np.NaN)
            else:
                Res.append(Reserved.text)  

            if Type is None:
                Ty.append(np.NaN)

            else:
                Ty.append(Type.text) 


            if km is None:
                 range1.append(np.NaN)
            else:
                range1.append(km.text)
                
            if cost is None:
                price.append(np.NaN)
            else:
                price.append(cost.text)

            if payment is None:
                paym.append(np.NaN)
            else:
                paym.append(payment.text)

            if Brand is None:
                Bran.append(np.NaN)
            else:
                Bran.append(Brand.text)
        
                

            
                
            pagenum.append(i)
        print('Page {} completed in {} seconds'.format(i, time.time()-start_time))

In [ ]:
len(productname)
len(price)
len(range1)
len(pagenum)
len(loc)

# Datacleaning using Pandas

In [ ]:
# converting into Datafrme and column names are provided

cars_df=pd.DataFrame({'Cityname':loc,'ProductName':productname,'Reserved':Res, 'Type':Ty,'Price':price,'Range':range1, 'payment':paym, 'Pagenumber':pagenum})
cars_df

In [ ]:
# Converted to CSV file
cars_df.to_csv('./cars_df.csv')

In [ ]:
cars_df1=pd.read_csv("./cars_df.csv")

In [ ]:
cars_df1

In [ ]:
#Dropped the column Unnamed and pasted the dataframe in new variable
cars_df2=cars_df1.drop("Unnamed: 0",axis=1)

In [ ]:
cars_df2

In [ ]:
# Splited the column "price" into two columns and store it variable called "new"
new=cars_df2["Price"].str.split("₹",n=2,expand=True)

In [ ]:
new

In [ ]:
# Creating two new columns in cardf2 and storing values from new dataframe
cars_df2['Pricepermonth']=new[1]
cars_df2['Totalprice']=new[2]

In [ ]:
# Converting the column into string column
cars_df2['Pricepermonth']=''+cars_df2['Pricepermonth'].astype(str)

In [ ]:
cars_df2['Totalprice']='₹'+cars_df2['Totalprice'].astype(str)

In [ ]:
#Removing textual data from the Range column and storing it in new variable
cars_df2['Range1']=cars_df2['Range'].str.extract('(\d{2,}(,)\d{2,}( )[a-z]{1,})',expand=True)

In [ ]:
#To get the textual data from the column Range
cars_df2['Fuel']=cars_df2['Range'].str.extract('(\d[a-z]{1,}( )[A-Za-z]{,5})',expand=True)

In [ ]:
# Splitted the textual data and divided into two new columns m, n
cars_df2[['m','n']]=cars_df2['Range'].str.split('r',n=1,expand=True)

In [ ]:
# Dropping the Range column and m column
cars_df2.drop(['Range','m'],axis='columns',inplace=True)

In [ ]:
#Renaming the Range1 column to Range for user understandability
cars_df2.rename(columns={'Range1':'Range'},inplace=True)

In [ ]:
#Splitting the Productname column at the spaces to make into three columns
cars_df2[['Year','Company_name','Product_version']]=cars_df2['ProductName'].str.split(' ',n=2,expand=True)

In [ ]:

cars_df2[['Company_name','n']]=cars_df2['Product_version'].str.extract('([A-Za-z]{1,}( ))',expand=True)
cars_df2

In [ ]:
#Dropping the empty colunm n
cars_df2.drop(['n'],axis='columns',inplace=True)

In [ ]:
# this will show true or false in Type column if "Automatic"/"Manual" is presnet
cars_df2['Type1']=cars_df2['Type'].str.contains("Automatic|Manual")==True

In [ ]:
cars_df2

In [ ]:
cars_df2['Product_version']=cars_df2['Product_version'].str.split(' ',n=2,expand=True)

In [ ]:
#Adding "Automatic"/"Manual" in the "Type1" column

substr=('Automatic','Manual')

cars_df2['Type1'] = cars_df2['Type'].str.findall('|'.join(substr)).str[0]



In [ ]:
cars_df2

In [ ]:
#Changing column names of "product version" & "Type1"
cars_df2.rename(columns={'Product_version':'Vehicle_version','Type1':'Mode'},inplace=True)

In [ ]:

#cars_df2[['Pricepermonth','Range']]=df["Pricepermonth", "Range"].str.replace(",","")
#cars_df2

In [ ]:
cars_df2['Fuel']=cars_df2['Range'].str.extract('(\d[a-z]{1,}( )[A-Za-z]{,5})',expand=True)

In [ ]:
# Removing /month from "Pricepremonth" column and replacing commas
cars_df2["Pricepermonth"] = cars_df2["Pricepermonth"].apply(lambda x:str(x).replace("/month","").replace(",","")).astype(float)

In [ ]:
cars_df2["Reserved"]

In [ ]:
#Removing commas from Total price column and converting it into float type
cars_df2["Totalprice"] = cars_df2["Totalprice"].apply(lambda x:str(x).replace(",","")).astype(float)

In [ ]:
cars_df2['Ownertype'].unique()

In [ ]:
#Extracting only number and text
cars_df2['Ownertype']=cars_df2['Ownertype'].str.extract('(\d[a-z]{1,3})',expand=True)

In [ ]:
# Dropping the columns
#Payment column is deleted bcz, for every car there is only zero down payment
cars_df2.drop(['ProductName','Reserved','Price','Payment','Pagenumber','pricepermonth'],axis='columns',inplace=True)

In [ ]:
Splitting into two columns at Manual/Automatic
cars_df2[['Type_name','Type1']]=cars_df2["Type"].str.split("Manual|Automatic",n=-1,expand=True)

In [ ]:
cars_df2

In [ ]:
#cars_df2.drop(['Range'],axis='columns',inplace=True)

In [ ]:
#Changing the column name
cars_df2['km_driven'] = cars_df2['Range']

In [ ]:
cars_df2

In [ ]:
#pd.set_option('display.max_rows', None)

In [ ]:
df = cars_df2.copy()

### City name

In [ ]:
df['Cityname'].describe()

In [ ]:
df['Cityname'].value_counts()

In [ ]:
cars_df2

In [ ]:
#copying the dataframe to cars24 variable
cars24=cars_df2.copy()

In [ ]:
#Printing first 5 rows
cars24.head()

In [ ]:
print(cars24.dtypes)

In [ ]:
#To know is there any Nans presnet in the columns
print("Cityname:", cars24['Cityname'].isna().sum())
print("Pricepermonth:", cars24['Pricepermonth'].isna().sum())
print("Totalprice:", cars24['Totalprice'].isna().sum())
print("Ownertype:", cars24['Ownertype'].isna().sum())
print("Fuel:", cars24['Fuel'].isna().sum())
print("Year:", cars24['Year'].isna().sum())
print("Vehicle_version:", cars24['Vehicle_version'].isna().sum())
print("Company_name:", cars24['Company_name'].isna().sum())
print("Mode:", cars24['Mode'].isna().sum())
print("Type_name:", cars24['Type_name'].isna().sum())
print("km_driven:", cars24['km_driven'].isna().sum())

In [ ]:
# To count the NaN values for every column by single line of code
cars24.isna().sum()

In [ ]:
cars24.shape

In [ ]:
#Dropping all the NAN value rows, by taking reference from price per month column
cars24=cars24.dropna(subset=['Pricepermonth'])

In [ ]:
cars24.shape

In [ ]:
# Filling it to Unknown
cars24=cars24.fillna({'Mode':'Unknown'})

In [ ]:
cars24

In [ ]:
cars241=cars24.copy()
cars241

In [ ]:
#To fill the NAN values which are present in KM_DRIVEN to the mean values, by taking average for every year in kmdriven
cars24['km_driven'] = cars24['km_driven'].fillna(cars24.groupby('Year')['km_driven'].transform('mean'))
cars24

In [ ]:
cars24['Fuel'].unique()

In [ ]:
# To replaace the text of "d Owner"
cars24["Fuel"] = cars24["Fuel"].apply(lambda x:str(x).replace("d Owner",""))

In [ ]:
cars24

In [ ]:
# Converting it into xlsx file
filename='cars24.xlsx'
cars24.to_excel(filename)

In [ ]:
cars24.head()

In [ ]:
# Reading xlsx file
cars24=pd.read_excel('cars24.xlsx',index_col=0)
cars24

In [ ]:
cars24.head()
cars24.shape

In [ ]:
#To know the frequency of texts in the Mode column
cars24["Mode"].value_counts()

In [ ]:
# Replacing "UNknowns" to "Manual" because they are high in number when compared to "Automatic"
cars24["Mode"] = cars24["Mode"].apply(lambda x:str(x).replace("Unknown","Manual"))

In [ ]:
cars24["Vehiclename"]=cars24["Company_name"]+" "+cars24["Vehicle_version"]

In [ ]:
cars24

## Exploratory Data Analysis



### Univariate plots

In [ ]:

fig=plt.figure(figsize=(15,5))

k=sns.countplot(x='Company_name',data=cars24)

y=k.set_xticklabels(k.get_xticklabels(), rotation=40, ha="right")
#ax.set_xticklabels(ax.get_xticklabels(), fontsize=7)
y=plt.title("To count the company cars which are there in the cars24 website",fontsize=20,fontweight="bold")
plt.tight_layout()
plt.show(y)

##### Insight
* Here the count plot illustrates that the most cars which are for sell is from "Maruti" company which is 1200+ cars among 9 cities and the second largest cars for selling is from Hyundai and remaining all are <200 in number

In [ ]:
#plt.figure(dpi=100)

fig=plt.figure(figsize=(15,5))


fig=sns.set(style="white")

fig=sns.histplot(data=cars24, x="Pricepermonth", kde=True,color='green',hue='Ownertype')
fig=fig.set_title("To count the frequency of price per month with owner type",fontsize=20,fontweight="bold")
fig=plt.tight_layout()
plt.show(fig)

##### Insight
* From this histogram we can tell that the peak pricepermonth for ownertype 1 and 2 is around 15,000, Furthermore we can tell that the 3rd,4th,5th,6th owner types are almost negligible in cars 24 company. Hence, we can tell that 99% of cars which come for cars24 company is of Ownertype 1 & 2.

In [ ]:
k=plt.figure(figsize=(10,7))

explode = (0.05, 0.05, 0.05, 0.05, 0.05,0.05,0.05,0.05,0.05)

k=cars24.groupby('Cityname').size().plot(kind='pie', autopct='%.2f',explode=explode)
#k=plt.pie(cars24['Cityname'].value_counts(),labels=cars24['Cityname'])

centre_circle = plt.Circle((0, 0), 0.70, fc='white')
k=fig = plt.gcf()
k=fig.gca().add_artist(centre_circle)
#fig=fig.set_title("Citynames with their percentages",fontsize=20,fontweight="bold")
k=plt.title("Citynames with their percentages of cars available in each city ",fontsize=12,fontweight="bold")
plt.show(k)

##### Insight
* From the above circle graph, it depicts that from cities like "Delhi", "Chennai","Bangalore","Ahmadabad", "Pune", "Mumbai" the same percentage of cars are coming for sale, However, the less percentage of cars coming for sale is from city "Gurgaon".Hyderabad and Noida occupies in between 8 to 4%

### Bivariate plots

In [ ]:

fig=plt.figure(figsize=(15,5))

k=sns.countplot(x='Year',data=cars24,hue="Fuel");
y=k.set_xticklabels(k.get_xticklabels(), rotation=40, ha="right")
#ax.set_xticklabels(ax.get_xticklabels(), fontsize=7)
k=plt.title("To count the fuel types per each year ",fontsize=15,fontweight="bold")
plt.tight_layout()
plt.show(y)

##### Insight
* From the above bar graph it shows that the highest number of cars came for the sale is in the year of 2017 for "Petrol" and "Diesel" cars and almost nill in the months of 2002 & 2004. Additionlly, the graph says that the cars with Fuel like "Petrol+CNG" and "Petrol+LPG" is very less in every year for sale

In [ ]:

k=plt.figure(figsize=(15,5))
k=sns.boxplot( x=cars24["Company_name"], y=cars24["Totalprice"])
y=k.set_xticklabels(k.get_xticklabels(), rotation=90, ha="right")
k=plt.title("To get the Avg Total price for every company car and their deviations from the average",fontsize=15,fontweight="bold")


##### Insight
* The above box plot tells us that the average total price, maximum and minimum prices for each car company in cars 24 site.

In [ ]:

k=plt.figure(figsize=(20,10))
k = sns.violinplot(x="Company_name", y="km_driven", hue="Mode",
                    data=cars24, palette="muted")
y=k.set_xticklabels(k.get_xticklabels(), rotation=40, ha="right")
k=plt.title("Maximum km_driven vs company name w.r.t Gear transmission",fontsize=15,fontweight="bold")


##### Insight
* The above violin plot tells that the Km_driven by each car company. Violin plot mosly uses for to show maximums. From the above plot it tells that "Toyata" company car driven most kilometers in "Automatic" and "Manual" modes

In [ ]:
k=plt.figure(figsize=(20,7))
k=sns.relplot(x="Totalprice", y="km_driven", hue="Mode", 
           data=cars24);
k=plt.title("Total price vs km_driven w.r.t Gear transmission",fontsize=15,fontweight="bold")



##### Insight
* From the above relation plot, it shows that for "Manual" type  85% of cars in the range of 0 to 1000000/- "Total price"
and km driven is in between 0 to 200000, However, in "Automatic" mode the 75% cars lies in between the price of 1000000 and
2000000 and Km_driven is in between 0 to 150000

In [ ]:
fig=plt.figure(figsize=(15,5))
k=sns.swarmplot(x='Cityname',y="km_driven",hue="Ownertype",data=cars24)

y=k.set_xticklabels(k.get_xticklabels(), rotation=40, ha="right")
#ax.set_xticklabels(ax.get_xticklabels(), fontsize=7)
k=plt.title("Cityname vs km_driven w.r.t Owner type",fontsize=15,fontweight="bold")
plt.tight_layout()
plt.show(y)

##### Insight
* The above swarn plot tells that 70% of cars in every city is of type 1st and 25% of cars is of 2nd type and 3% is of 3rd type 
an remaining 2% is of remaining types

In [ ]:
top_10car = cars24.groupby("Vehicle_version")['Totalprice'].max().nlargest(10)
top10 = pd.DataFrame(top_10car)

In [ ]:
top10_car = pd.merge(top10,cars24,on = "Totalprice")

In [ ]:
top10_car


In [ ]:
Top=cars24.groupby(["Cityname"])["Totalprice","km_driven","Company_name","Vehicle_version","Ownertype","Fuel"].max()
Bottom=cars24.groupby(["Cityname"])["Totalprice","km_driven","Company_name","Vehicle_version","Ownertype","Fuel"].min()

In [ ]:
Top=pd.DataFrame(Top)

In [ ]:
Top=Top.reset_index()
Top

In [ ]:
Bottom=pd.DataFrame(Bottom)

In [ ]:
Bottom=Bottom.reset_index()
Bottom

In [ ]:
Top["Vehiclename"]=Top["Company_name"]+" "+Top["Vehicle_version"]
Bottom["Vehiclename"]=Bottom["Company_name"]+" "+Bottom["Vehicle_version"]

In [ ]:
#k=plt.figure(1)
#k=plt.figure(dpi=120)
k=plt.figure(figsize=(15,5))
k=plt.subplot(1, 2, 1)
k=sns.countplot(Top["Cityname"],hue=Top["Vehiclename"])
k=plt.xticks(rotation=90)
k=plt.legend(loc='upper left')
k=plt.title("To count the top cars in every city wrt Vehiclename",fontsize=10,fontweight="bold")
k=plt.tight_layout()


k=plt.subplot(1, 2, 2)
k=sns.countplot(Bottom["Vehiclename"],hue=Bottom["Fuel"])
k=plt.xticks(rotation=45)
k=plt.title("To count the bottom cars in every city wrt fueltype",fontsize=10,fontweight="bold")
k=plt.tight_layout()


#k=plt.figure(2)
#k=plt.figure(dpi=120)
k=plt.figure(figsize=(15,5))
k=plt.subplot(2, 2, 1)
k=sns.countplot(Top["Ownertype"])
k=plt.xticks(rotation=45)
k=plt.title("To count the top cars owner type",fontsize=10,fontweight="bold")
k=plt.tight_layout()

k=plt.subplot(2, 2, 2)
k=sns.countplot(Bottom["Ownertype"])
k=plt.xticks(rotation=45)
k=plt.title("To count the Bottom cars owner type",fontsize=10,fontweight="bold")
k=plt.tight_layout()


k=plt.subplot(2, 2, 3)
k=sns.countplot(cars24["Ownertype"])
k=plt.xticks(rotation=45)
k=plt.title("To count cars owner type available in cars 24 data set",fontsize=10,fontweight="bold")
k=plt.tight_layout()


plt.show(k)


##### Insight
1. First bar graph tells that those four vehicle names have the maximum total price with respective states.
2. Second count plot tells that the vehicles which have minimum Total Price is of fuel "Diesel"
3. Third bar plot tells that the count of maximum price vehicles respective with Ownertype. 4th owner type car has 4 cars 
    as maximum price
4. Fourth plot tells that the 1st owner type cars has minimum price
5. Fifth count plot tells that the number of vehicles in each owner type, 1st: 2000+, 2nd: 500+, 3rd: <100 and 4th, 5th, 6th    are  almost negligible.

In [ ]:

k=plt.figure(figsize=(15,5))
k=plt.plot(cars24.groupby(["Cityname"])["Totalprice"].max())
k=plt.plot(cars24.groupby(["Cityname"])["Totalprice"].min())
k=plt.legend(['Maximum', 'Minimum'])
k=plt.title("Maximum & Minimum price of cars in each & every city",fontsize=15,fontweight="bold")
plt.show(k)

##### Insight
* The above line graph illustrates that the maximum and Minimum "Total price" in each "City"

In [ ]:
k=plt.figure(dpi = 100)
k=sns.heatmap(cars24.corr(),linecolor='white',linewidths=2,annot=True);
k=plt.title("To show correlation b/w numerical variables",fontsize=15,fontweight="bold")

##### Insight
* The above heat map tells that there is 15%negative correlation between pricepermonth/Totaprice and km_driven. It represents that as 
    one increases other decreases
*The above heat map tells that there is 50%negative correlation between pricepermonth/Totaprice and year. It represents that as 
    one increases other decreases
* The above heat map tells that there is 45%positive correlation between pricepermonth/Totaprice and year. It represents that as 
    one increases other increase
    

In [ ]:
# Latest year cars in every city
k=plt.figure(figsize=(15,5))
k=plt.subplot(1, 2, 1)
k=sns.swarmplot(x="Vehiclename", y="Totalprice", hue="Cityname", data=cars24[cars24["Year"]==cars24["Year"].max()])
k=plt.xticks(rotation=90)
k=plt.legend(loc='upper right')
k=plt.title("Vehiclename vs Total price wrt cityname",fontsize=13,fontweight="bold")
k=plt.tight_layout()

k=plt.subplot(1, 2, 2)
k=sns.swarmplot(x="Vehiclename", y="Cityname", hue="Ownertype", data=cars24[cars24["Year"]==cars24["Year"].max()],marker='D',s=8)
k=plt.xticks(rotation=90)
k=plt.legend(loc='upper left')
k=plt.title("Vehiclename vs Cityname wrt Ownertype",fontsize=13,fontweight="bold")
k=plt.tight_layout()

k=plt.figure(figsize=(15,5))
k=plt.subplot(2, 2, 1)
k=sns.swarmplot(x="Vehiclename", y="Mode", hue="Fuel", marker="*", data=cars24[cars24["Year"]==cars24["Year"].max()],s=10)
k=plt.xticks(rotation=90)
k=plt.legend(loc='center')
k=plt.title("Vehiclename vs Gear transmission wrt fueltype",fontsize=13,fontweight="bold")
k=plt.tight_layout()

##### Insight
   *This swarn plot showing that latest cars and their prices in every city. From this graph, it shows that eventhough same 
    cars but showing differences in "Total prices" in between cities.
   
   *2nd graph is showing that all the latest cars belongs to 1st car ownertype, that means nobody interested to sell the cars
    for 2nd and 3rd owner type
   
   *3rd graph representing that 90% of cars are "Manual" type and only two cars belongs to automatic and all the cars which 
    came for sale all are belongs to petrol cars. From this, the interpretation will be people are not willing to sell 
    "Diesel/CNG" cars
   

### Multivariateplot

In [ ]:
k=sns.pairplot(cars24,hue="Mode");
#k=plt.title("Pair plot between numerical variables wrt Gear transmission",fontsize=10,fontweight="bold")
plt.show(k)

##### Insight
* The above plot tells us that the relation between all the numerical variables present in the dataset

In [ ]:
# To install modules in jupyter note book instead of command prompt we need to use exclamation mark before pip
! pip install ipympl

In [ ]:
#For importing 3d plot libraries

from mpl_toolkits import mplot3d

k=plt.figure(figsize=(5,10))
#fig = plt.figure()
ax = plt.axes(projection = '3d')
ax.scatter(cars24["km_driven"],cars24["Totalprice"],cars24["Pricepermonth"])
k=plt.xticks(rotation=40)
k=plt.tight_layout()
k=plt.title("3D plot km_driven vs Totalprice vs Price per month ",fontsize=10,fontweight="bold")
plt.show(k)

In [ ]:


cars24.groupby(["Fuel","Ownertype"])["Pricepermonth"].mean()

In [ ]:
cars24.groupby(["Fuel","Ownertype"])["Totalprice"].mean()

In [ ]:
cars24.head()

## Some Assumptions

#### Applicable for every class:
   - if we drive Minimum  40000km then we can be advantegous with diesel cars over Petrol cars, bcz initial cosst of Diesel car
     is high
   - if we want to change the car for 2-3 years then petrol cars would be better, because initial cost is hiigher for diesel
     cars. Diesel car will be advantegeous if we maintain 5 to 10 years.
   - Another advantage of Diesel cars is resale price will be higher because less number of cars and it gives good pickup also.
   - Better if we take diesel/CNG car because it gives 21km/lr & 16km/lr milage and also gives good accelaration
   - Mode is "Automatic" for younger people because they like that most and gives smoothness in drive in traffic 
     conditions, but for the old people and who is confortable with clutch they like Manual because they habituated with it
   - For long drives it is suitable to have "Automatic" transmission, otherwise it may give fatigueness to the driver
   - The insurance cost would be higher for diesel cars
   - The price per months taken by taking mean values of price per month for each fuel type and owner type.

 ######  For  middle class the ideal parameters we can consider is:
       1. Total price is 7 to 8 lakhs for 1st ownertype and Total price is 4 to 6 lakhs for 2nd ownertype for Diesel cars
       2. Total price is 5 to 5.5 lakhs for 1st ownertype and Total price is 2 to 4 lakhs for 2nd ownertype for Petrol cars
       3. Pricepermonth(EMI)  is 16000 to 18000  for 1st ownertype and Pricepermonth  is 13000 to 15000 for Diesel
          cars for 2nd ownertype for Diesel cars
       4. Pricepermonth((EMI)  is 11000 to 13500  for 1st ownertype and Pricepermonth  is 9500 to 11000 for Petrol
          cars for 2nd ownertype  for Petrol cars
       
  ######  For  rich class the ideal parameters we can consider is:
        1. Total price is 12 to 18 lakhs for 1st ownertype and Total price is 8 to 12 lakhs for 2nd ownertype for Diesel cars
        2. Total price is 10 to 15 lakhs for 1st ownertype and Total price is 7 to 11 lakhs for 2nd ownertype for Petrol cars
        3. Pricepermonth(EMI)  is 26000 to 40000  for 1st ownertype and Pricepermonth  is 21000 to 30000 for Diesel
           cars for 2nd ownertype for Diesel cars
        4. Pricepermonth(EMI)  is 20000 to 37000  for 1st ownertype and Pricepermonth  is 15000 to 27000 for petrol
           cars for 2nd ownertype for Petrol cars
           
         
- For CNG cars the prices is between Petrol & Diesel cars, here LPG cars only 2 cars so they taken out

In [ ]:
city_name = list(input("Please enther the city name: ").split(","))
years=int(input("Enter the starting year: "))
# You can enter zero if you dont know number of Kilometers driven
kiometersdriven=float(input("Enter the no of kilometers that car already travelled: "))
Ownertype=list(input("please enter the owner type: ").split(","))
Mode=list(input("Please enther the mode : ").split(","))
companyname = list(input("Please enther the company name: ").split(","))
# You can enter Startprice and Endprice only if you want buy the car otherwise put "0" in both Start&End price
Startprice=int(input("Minimum price you can put: "))
Endprice=int(input("Maximum price you can put: "))
# You can enter Startpricemonth and Endpricemonth only if you want buy the car for a month otherwise put "0" in both Start&End pricepermonth
Startpricemonth=int(input("Minimum pricepermonth(for EMI) you can put: "))
Endpricemonth=int(input("Maximum pricepermonth(for EMI) you can put: "))
# Any_df means any car type
Any_df=cars24[(cars24["Cityname"].isin(city_name)) & (cars24["Year"]>=years) & (cars24['km_driven']>=kiometersdriven) & (cars24["Ownertype"].isin(Ownertype)) & (((cars24["Totalprice"]>=Startprice) & (cars24["Totalprice"]<Endprice)) | ((cars24["Pricepermonth"]>=Startpricemonth) & (cars24["Pricepermonth"]<=Endpricemonth))) & (cars24["Mode"].isin(Mode)) & ((cars24["Company_name"].isin(companyname)) | (~cars24["Company_name"].isin(companyname)))]
Any_df
Any_df.shape
k=plt.figure(figsize=(15,5))

k=plt.subplot(1, 2, 1)
k = sns.barplot(x="Vehiclename", y="Totalprice", hue="Fuel",data=Any_df, palette="muted")
k=k.set_xticklabels(k.get_xticklabels(), rotation=90, ha="right")
k=plt.legend(loc='upper right')
k=plt.title("Vehiclename vs Totalprice wrt fueltype",fontsize=10,fontweight="bold")
k=plt.tight_layout()

k=plt.subplot(1, 2, 2)
k=sns.barplot(x="Vehiclename",y="km_driven", hue="Ownertype",data=Any_df, palette="muted")
k=k.set_xticklabels(k.get_xticklabels(), rotation=90, ha="right")
k=plt.legend(loc='upper right')
k=plt.title("Vehiclename vs km_driven wrt Ownertype",fontsize=10,fontweight="bold")
k=plt.tight_layout()

k=plt.figure(figsize=(15,10))
k=plt.subplot(2, 2, 1)
k=sns.barplot(x="Vehiclename",y="Totalprice",hue="Cityname",data=Any_df)
k=k.set_xticklabels(k.get_xticklabels(), rotation=90, ha="right")
k=plt.legend(loc='upper right')
k=plt.title("Vehiclename vs Totalprice wrt Cityname",fontsize=10,fontweight="bold")
k=plt.tight_layout()

k=plt.subplot(2, 2, 2)
k = sns.swarmplot(x="Vehiclename", y="Pricepermonth", hue="Mode",data=Any_df, palette="muted")
k=k.set_xticklabels(k.get_xticklabels(), rotation=90, ha="right")
k=plt.legend(loc='upper right')
k=plt.title("Vehiclename vs Price per month  wrt Gear transmission",fontsize=10,fontweight="bold")
k=plt.tight_layout()

plt.show(k)


In [ ]:
#Partcomp means For Particular company
Partcomp_df=cars24[(cars24["Cityname"].isin(city_name)) & (cars24["Year"]>=years) & (cars24['km_driven']>=kiometersdriven) & (cars24["Ownertype"].isin(Ownertype)) & (cars24["Totalprice"]>=Startprice) & (cars24["Totalprice"]<=Endprice) & (cars24["Mode"].isin(Mode)) & ((cars24["Company_name"].isin(companyname)))]
Partcomp_df

#### Insight

##### For Delhi city and car owner type is 1st
    - for middle class in Petrol cars budget and to buy permanently
       - "Maruti Baleno DELTA DDIS 190" with Total price 549299/- is good for Middle class people because its available as 
         Diesel fuel in petrol cars budget and also it is Manual type. For middle class young people "Tata Tiago XZA 1.2 
         REVOTRON" with Total price 542299 is good, beacause it is automatic and also Diesel car

    - for middle class in Petrol cars budget and to buy on EMI basis(pricepermonth)
       - Best car in Manual transmission is "Maruti Dzire", with price per month 12012/- and Fueltype is Diesel.  For middle 
       class young people "Tata Tiago XZA 1.2 REVOTRON" with priceper
       month 11970/- is good, beacause it is automatic and also Diesel car

 
    - for rich class in Diesel cars budget and to buy permanently
       - "Honda civic ZX MT DIESEL" with Total price 1751899/- is good for rich class people because its available as Diesel 
       fuel in Diesel cars budget and also it is Manual type. For rich class young people "Toyota Innova Crysta Touring 
       Sport Diesel AT" with Total price 1547299/- is good, beacause it is automatic and also Diesel car
          
     - for rich class in Diesel cars budget and to buy on EMI basis(pricepermonth)
       - Best car in Manual transmission is "Honda civic ZX MT DIESEL", with price per  month 38970/- and Fueltype is Diesel
       for rich class young people "Toyota Innova Crysta Touring SportDiesel AT" with pricepermonth 34419/- is good, 
       beacause it is automatic and also Diesel car


 
 
 
 

### Conclusion
To summarize this, from the above all the graphs, first, we can tell that the "Maruti" company cars are high in number for 
sale. Second, 70% of cars coming for sale is of 1st owner type, Third, the maximum price company cars for sale are Volvo
and Volkswegan. fourth, manual cars 80% higher than automatic cars. fifth, the car prices will be different for different cities
depends on car ownertype, year of purchase, city average costs. sixth, To get good mileage people will prefer Diesel/CNG cars than petrol cars. Seventh, the 90 % cars coming for sale are in between 0 to 10000km driven. Eigth, Petrol cars are more than Diesel cars. Nineth, the price per month is around 0 to 20000  and Total price is 2 to 30 lakhs, for almost all of the cars.
Tenth, 90% of cars year of purchase is in between 2010 to 2020.These are some of the points to consider based on the above data.